In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pathlib import Path

In `config`:

    config = Config(
        max_length = 64  # in tokens
        from_pretrained = "facebook/bart-base"
        model_type = "Twin"
    )

In [4]:
from smtag.config import config
assert config.max_length==64 and config.from_pretrained=='facebook/bart-base' and config.model_type=='VAE'

In [5]:
from transformers import __version__
__version__

'4.15.0'

## Extracting examples for LM

In [6]:
from smtag.extract import ExtractorXML

In [7]:
! dir /data/xml/oapmc_articles

eval  test  train


In [12]:
# ! rm -fr /data/text/oapmc_title

In [13]:
corpus = "/data/xml/oapmc_articles"
text_examples = "/data/text/oapmc_title"
xpath = ".//article-meta/title-group/article-title"
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [14]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

/data/text/oapmc_title created


In [15]:
extractor_lm.extract_from_corpus()

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


{PosixPath('/data/text/oapmc_title/train.txt'): 967335,
 PosixPath('/data/text/oapmc_title/eval.txt'): 2970,
 PosixPath('/data/text/oapmc_title/test.txt'): 3004}

#### same via CLI:

```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

## Preparing tokenized dataset for LM

In [16]:
from smtag.dataprep import PreparatorLM

In [6]:
tokenized_examples = "/data/json/oapmc_title"

In [18]:
# ! rm -fr /data/json/oapmc_title

In [19]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

/data/json/oapmc_title created


In [20]:
prep_lm.run()

Preparing: train


100%|██████████| 967335/967335 [22:17<00:00, 723.36it/s]



Length verification: OK!

average input_ids length = 35 (min=16, max=64) tokens
longest example: <s>Efficiency and Tolerability of Induction and Consolidation Therapy with Arsenic Trioxide/Bortezomib/Ascorbic Acid/Dexamethasone (ABCD) Regimen Compared to Bortezomib/Dexamethasone (BD) Regimen in</s>
shortest example: <s>Data on entrepreneurship education and entrepreneurial performance of aspiring entrepreneurs in selected Nigerian universities</s>
Preparing: eval


100%|██████████| 2970/2970 [00:02<00:00, 1009.73it/s]



Length verification: OK!

average input_ids length = 35 (min=18, max=64) tokens
longest example: <s>Characterization of an Indole-3-Acetamide Hydrolase from Alcaligenes faecalis subsp. parafaecalis and Its Application in Efficient Preparation of Both Enantiomers of Chiral Building Block 2,3-Dihydro-1,4-Benz</s>
shortest example: <s>A bilateral tumor model identifies transcriptional programs associated with patient response to immune checkpoint blockade</s>
Preparing: test


100%|██████████| 3004/3004 [00:04<00:00, 747.14it/s]


Length verification: OK!

average input_ids length = 35 (min=18, max=64) tokens
longest example: <s>Comparison of Inappropriate Shocks and Other Health Outcomes Between Single- and Dual-Chamber Implantable Cardioverter-Defibrillators for Primary Prevention of Sudden Cardiac Death: Results From the Cardiovascular Research Network Longitudinal Study of Implantable Cardioverter-Def</s>
shortest example: <s>Primary care provider perceptions of intake transition records and shared care with outpatient cardiac rehabilitation programs</s>


same vie CLI:
    
```bash
python -m smtag.cli.lm.dataprep /data/text/mini /data/json/mini
```

## Train LM

In [7]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

In [8]:
no_cache = True
loader_path = "./smtag/loader/loader_lm.py"
data_config_name = "SEQ2SEQ"
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide
model_type = "VAE"
from_pretrained = config.from_pretrained

In [9]:
tokenizer.mask_token = '<mask>'  # why is this here? maybe because in case of character-level tokenizer
# tokenizer.unk_token = '<unk>'

Model architecture parameters: 

```python
model_config = VAEConfigLM(
    freeze_pretrained=None, #'encoder',
    hidden_features=256,
    z_dim=2048,
    gamma=1E-4,  # weight of lm loss as compared to z_loss
    sampling_iterations=1000,
    seq_length=config.max_length,
    residuals=False,
)
```

In [11]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 1,
    logging_steps = 100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)
training_args_tokcl

TrainingArgumentsLM(output_dir='/lm_models', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/lm_models/runs/Apr10_13-35-13_ea25f0d9a7eb', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=100, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, seed=42, bf16

In [12]:
train_lm(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

tokenizer vocab size: 50265

Loading datasets found in /data/json/oapmc_title.
using ./smtag/loader/loader_lm.py as dataset loader.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset bio_lang downloaded and prepared to /cache/bio_lang/SEQ2SEQ-1ddd9282e7145ba8/0.0.1/ff9dbe678aba55520bef244a55868bd4acf85eb086ce21a8454da23294256c2e. Subsequent calls will reuse this data.


...

INFO:***** Running Evaluation *****
INFO:  Num examples = 2970
INFO:  Batch size = 32




<s>Caenorhabditis elegans Histone Methyltransferase MET-2 Tracks[Shields] the Male X Chromosome from Checkpoint Machinery and Mediates Meiotic Sex Chromosome Inactivation</s>




INFO:***** Running Evaluation *****
INFO:  Num examples = 2970
INFO:  Batch size = 32




<s>Markers of type I collagen degradation and synthesis in the monitoring of treatment response in bone metastases from breast carcinoma.</s>




KeyboardInterrupt: 